In [1]:
!pip install openpyxl
import os
import sys
import pandas as pd

# Add root directory of the project to system.path
sys.path.append(os.path.abspath('..'))

from db.db_utils import get_engine, insert_dataframe_to_db, run_query
from dotenv import load_dotenv


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Load environement variables
load_dotenv()

True

#### Load the raw data (CSV/Excel) into a DataFrame

In [3]:
# Load the dataset
df_raw = pd.read_excel('../data/raw/NHSOF_2.3.i_I00708_D.xlsx', sheet_name='Indicator data', skiprows=14)
df_raw.head()

,Year,Quarter,Period of coverage,Breakdown,Level,Level description,Indicator value,Lower CI,Upper CI,Standardised ratio,Standardised ratio lower CI,Standardised ratio upper CI,Observed,Population,Expected,Percent unclassified
0,2023/24,Annual,1/4/2023 to 31/3/2024,England,England,England,841.8,839.5,844.1,102.6,102.3,102.9,521065,57690300,507865.9,NaN
1,2022/23,Annual,1/4/2022 to 31/3/2023,England,England,England,780.8,778.6,783,95.2,94.9,95.4,476774,57106400,500993.5,NaN
2,2021/22,Annual,1/4/2021 to 31/3/2022,England,England,England,808.9,806.6,811.2,98.6,98.3,98.9,486173,56554900,493128.1,NaN
3,2020/21,Annual,1/4/2020 to 31/3/2021,England,England,England,662.2,660.2,664.3,80.7,80.5,81,397738,56550100,492767.4,NaN
4,2019/20,Annual,1/4/2019 to 31/3/2020,England,England,England,862.1,859.8,864.5,105.1,104.8,105.4,512608,56287000,487822.8,NaN


#### Saving the loaded data to new file

In [4]:
df_raw.to_csv('../data/processed/emergency_readmission_processed.csv', index=False)

#### Load Processed Data for Cleaning

In [5]:
df = pd.read_csv('../data/processed/emergency_readmission_processed.csv')

In [6]:
df.head()

,Year,Quarter,Period of coverage,Breakdown,Level,Level description,Indicator value,Lower CI,Upper CI,Standardised ratio,Standardised ratio lower CI,Standardised ratio upper CI,Observed,Population,Expected,Percent unclassified
0,2023/24,Annual,1/4/2023 to 31/3/2024,England,England,England,841.8,839.5,844.1,102.6,102.3,102.9,521065,57690300,507865.9,NaN
1,2022/23,Annual,1/4/2022 to 31/3/2023,England,England,England,780.8,778.6,783,95.2,94.9,95.4,476774,57106400,500993.5,NaN
2,2021/22,Annual,1/4/2021 to 31/3/2022,England,England,England,808.9,806.6,811.2,98.6,98.3,98.9,486173,56554900,493128.1,NaN
3,2020/21,Annual,1/4/2020 to 31/3/2021,England,England,England,662.2,660.2,664.3,80.7,80.5,81,397738,56550100,492767.4,NaN
4,2019/20,Annual,1/4/2019 to 31/3/2020,England,England,England,862.1,859.8,864.5,105.1,104.8,105.4,512608,56287000,487822.8,NaN


In [7]:
df.shape

(57639, 16)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57639 entries, 0 to 57638
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Year                         57639 non-null  object 
 1   Quarter                      57639 non-null  object 
 2   Period of coverage           57639 non-null  object 
 3   Breakdown                    57639 non-null  object 
 4   Level                        57639 non-null  object 
 5   Level description            57639 non-null  object 
 6   Indicator value              57639 non-null  object 
 7   Lower CI                     57639 non-null  object 
 8   Upper CI                     57639 non-null  object 
 9   Standardised ratio           57639 non-null  object 
 10  Standardised ratio lower CI  57639 non-null  object 
 11  Standardised ratio upper CI  57639 non-null  object 
 12  Observed                     57639 non-null  object 
 13  Population      

In [9]:
# Convert columns to numeric
cols_to_convert = [
    'Indicator value', 
    'Lower CI', 
    'Upper CI',
    'Standardised ratio', 
    'Standardised ratio lower CI',
    'Standardised ratio upper CI', 
    'Observed', 
    'Population', 
    'Expected']

for col in cols_to_convert:
    df_raw[col] = pd.to_numeric(df_raw[col], errors='coerce')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57639 entries, 0 to 57638
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Year                         57639 non-null  object 
 1   Quarter                      57639 non-null  object 
 2   Period of coverage           57639 non-null  object 
 3   Breakdown                    57639 non-null  object 
 4   Level                        57639 non-null  object 
 5   Level description            57639 non-null  object 
 6   Indicator value              57639 non-null  object 
 7   Lower CI                     57639 non-null  object 
 8   Upper CI                     57639 non-null  object 
 9   Standardised ratio           57639 non-null  object 
 10  Standardised ratio lower CI  57639 non-null  object 
 11  Standardised ratio upper CI  57639 non-null  object 
 12  Observed                     57639 non-null  object 
 13  Population      

In [11]:
df.columns

Index(['Year', 'Quarter', 'Period of coverage', 'Breakdown', 'Level',
       'Level description', 'Indicator value', 'Lower CI', 'Upper CI',
       'Standardised ratio', 'Standardised ratio lower CI',
       'Standardised ratio upper CI', 'Observed', 'Population', 'Expected',
       'Percent unclassified'],
      dtype='object')

In [12]:
df.isnull().sum()

Year                              0
Quarter                           0
Period of coverage                0
Breakdown                         0
Level                             0
Level description                 0
Indicator value                   0
Lower CI                          0
Upper CI                          0
Standardised ratio                0
Standardised ratio lower CI       0
Standardised ratio upper CI       0
Observed                          0
Population                        0
Expected                          0
Percent unclassified           8360
dtype: int64

In [13]:
df['Level description'].unique()[:10]

array(['England', 'Female', 'Male', '0 to 4', '5 to 9', '10 to 14',
       '15 to 18', '19 to 24', '25 to 29', '30 to 34'], dtype=object)

In [14]:
print('Unique Level description values: ', df['Level description'].nunique())

Unique Level description values:  438


In [15]:
sex_labels = ['Male', 'Female']
national_labels = ['England']
age_groups = [f'{i} to {i+4}' for i in range(0, 90, 5)] + ['90+']
deprivation_labels = ['1 - Most deprived', '10 - Least deplrived'] + [str(i) for i in range(2, 10)]

known_regions = ['Adur', 'Amber Valley', 'Arun', 'Ashfield', 'Ashford', 'Babergh', 'Barking and Dagenham', 'Barnet', 'Barnsley', 'Basildon',
                 'Basingstoke and Deane', 'Bassetlaw', 'Bath and North East Somerset', 'Bedford', 'Bexley', 'Birmingham','Blaby', 
                 'Blackburn with Darwen', 'Blackpool', 'London', 'Bolsover', 'Bolton', 'Boston', 'Bournemouth, Christchurch and Poole', 
                 'Bracknell Forest', 'Bradford', 'Braintree', 'Breckland', 'Brent', 'Brentwood', 'Brighton and Hove', 'Bristol, City of', 
                 'Broadland', 'Bromley', 'Bromsgrove', 'Broxbourne', 'Broxtowe', 'Buckinghamshire', 'Burnley', 'Bury', 'Calderdale', 
                 'Cambridge', 'Camden', 'Cannock Chase', 'Canterbury', 'Castle Point', 'Central Bedfordshire', 'Charnwood', 'Chelmsford', 
                 'Cheltenham', 'Cherwell', 'Cheshire East', 'Cheshire West and Chester', 'Chesterfield', 'Chichester', 'Chorley', 
                 'City of London', 'Colchester', 'Cornwall', 'Cotswold', 'County Durham', 'Coventry', 'Crawley', 'Croydon', 'Cumberland', 
                 'Dacorum', 'Darlington', 'Dartford', 'Derby', 'Derbyshire Dales', 'Doncaster', 'Dorset', 'Dover', 'Dudley', 'Ealing', 
                 'East Cambridgeshire', 'East Devon', 'East Hampshire', 'East Hertfordshire', 'East Lindsey', 'East Riding of Yorkshire', 
                 'East Staffordshire', 'East Suffolk', 'Eastbourne', 'Eastleigh', 'Elmbridge', 'Enfield', 'Epping Forest', 'Epsom and Ewell',
                 'Erewash', 'Exeter', 'Fareham', 'Fenland', 'Folkestone and Hythe', 'Forest of Dean', 'Fylde', 'Gateshead', 'Gedling', 
                 'Gloucester', 'Gosport', 'Gravesham', 'Great Yarmouth', 'Greenwich', 'Guildford', 'Hackney', 'Halton', 
                 'Hammersmith and Fulham', 'Harborough', 'Haringey', 'Harlow', 'Harrow', 'Hart', 'Hartlepool', 'Hastings', 'Havant', 
                 'Havering', 'Herefordshire, County of', 'Hertsmere', 'High Peak', 'Hillingdon', 'Hinckley and Bosworth', 'Horsham', 
                 'Hounslow', 'Huntingdonshire', 'Hyndburn', 'Ipswich', 'Isle of Wight', 'Isles of Scilly', 'Islington', 
                 'Kensington and Chelsea', "King's Lynn and West Norfolk", 'Kingston upon Hull, City of', 'Kingston upon Thames', 
                 'Kirklees', 'Knowsley', 'Lambeth', 'Lancaster', 'Leeds', 'Leicester', 'Lewes', 'Lewisham', 'Lichfield', 'Lincoln', 
                 'Liverpool', 'Luton', 'Maidstone', 'Maldon', 'Malvern Hills', 'Manchester', 'Mansfield', 'Medway', 'Melton', 'Merton', 
                 'Mid Devon', 'Mid Suffolk', 'Mid Sussex', 'Middlesbrough', 'Milton Keynes', 'Mole Valley', 'New Forest',
                 'Newark and Sherwood', 'Newcastle upon Tyne', 'Newcastle-under-Lyme', 'Newham', 'North Devon', 'North East Derbyshire', 
                 'North East Lincolnshire', 'North Hertfordshire', 'North Kesteven', 'North Lincolnshire', 'North Norfolk', 
                 'North Northamptonshire', 'North Somerset', 'North Tyneside', 'North Warwickshire', 'North West Leicestershire', 
                 'North Yorkshire', 'Northumberland', 'Norwich', 'Nottingham', 'Nuneaton and Bedworth', 'Oadby and Wigston', 'Oldham', 
                 'Oxford', 'Pendle', 'Peterborough', 'Plymouth', 'Portsmouth', 'Preston', 'Reading', 'Redbridge', 'Redcar and Cleveland', 
                 'Redditch', 'Reigate and Banstead', 'Ribble Valley', 'Richmond upon Thames', 'Rochdale', 'Rochford', 'Rossendale', 
                 'Rother', 'Rotherham', 'Rugby', 'Runnymede', 'Rushcliffe', 'Rushmoor', 'Rutland', 'Salford', 'Sandwell', 'Sefton', 
                 'Sevenoaks', 'Sheffield', 'Shropshire', 'Slough', 'Solihull', 'Somerset', 'South Cambridgeshire', 'South Derbyshire',
                 'South Gloucestershire', 'South Hams', 'South Holland', 'South Kesteven', 'South Norfolk', 'South Oxfordshire',
                 'South Ribble', 'South Staffordshire', 'South Tyneside', 'Southampton', 'Southend-on-Sea', 'Southwark', 'Spelthorne',
                 'St Albans', 'St. Helens', 'Stafford', 'Staffordshire Moorlands', 'Stevenage', 'Stockport', 'Stockton-on-Tees', 
                 'Stoke-on-Trent', 'Stratford-on-Avon', 'Stroud', 'Sunderland', 'Surrey Heath', 'Sutton', 'Swale', 'Swindon', 'Tameside', 
                 'Tamworth', 'Tandridge', 'Teignbridge', 'Telford and Wrekin', 'Tendring', 'Test Valley', 'Tewkesbury', 'Thanet', 
                 'Three Rivers', 'Thurrock', 'Tonbridge and Malling', 'Torbay', 'Torridge', 'Tower Hamlets', 'Trafford', 'Tunbridge Wells', 
                 'Uttlesford', 'Vale of White Horse', 'Wakefield', 'Walsall', 'Waltham Forest', 'Wandsworth', 'Warrington', 'Warwick', 
                 'Watford', 'Waverley', 'Wealden', 'Welwyn Hatfield', 'West Berkshire', 'West Devon', 'West Lancashire', 'West Lindsey', 
                 'West Northamptonshire', 'West Oxfordshire', 'West Suffolk', 'Westminster', 'Westmorland and Furness', 'Wigan', 
                 'Wiltshire', 'Winchester', 'Windsor and Maidenhead', 'Wirral', 'Woking', 'Wokingham', 'Wolverhampton', 'Worcester', 
                 'Worthing', 'Wychavon', 'Wyre', 'Wyre Forest', 'York', 'Allerdale', 'Barrow-in-Furness', 'Carlisle', 'Copeland', 'Craven', 
                 'Eden', 'Hambleton', 'Harrogate', 'Mendip', 'Richmondshire', 'Ryedale', 'Scarborough', 'Sedgemoor', 'Selby', 
                 'Somerset West and Taunton', 'South Lakeland', 'South Somerset', 'Corby', 'Daventry', 'East Northamptonshire', 'Kettering', 
                 'Northampton', 'South Northamptonshire', 'Wellingborough', 'Aylesbury Vale', 'Chiltern', 'South Bucks', 'Wycombe', 
                 'Bournemouth', 'Christchurch', 'East Dorset', 'Forest Heath', 'North Dorset', 'Poole', 'Purbeck', 'St Edmundsbury', 
                 'Suffolk Coastal', 'Taunton Deane', 'Waveney', 'West Dorset', 'West Somerset', 'Weymouth and Portland', 'Shepway',
                 'Cambridgeshire', 'Derbyshire', 'Devon', 'East Sussex', 'Essex', 'Gloucestershire', 'Hampshire', 'Hertfordshire', 'Kent',
                 'Lancashire', 'Leicestershire', 'Lincolnshire', 'Norfolk', 'Nottinghamshire', 'Oxfordshire', 'Staffordshire', 'Suffolk',
                 'Surrey', 'Warwickshire', 'West Sussex', 'Worcestershire', 'Cumbria', 'Northamptonshire', 'East Midlands', 'East of England',
                 'North East', 'North West', 'South East', 'South West', 'West Midlands', 'Yorkshire and The Humber']

known_conditions = ['Acute bronchitis', 'Angina pectoris', 'Asthma', 'Atrial fibrillation and flutter', 'Bronchiectasis',
                    'Chronic ischaemic heart disease', 'Chronic viral hepatitis B with delta-agent', 
                    'Chronic viral hepatitis B without delta-agent', 'Dementia in Alzheimer disease',
                    'Dementia in other diseases classified elsewhere', 'Emphysema', 'Epilepsy', 'Essential (primary) hypertension',
                    'Folate deficiency anaemia', 'Heart failure', 'Hypertensive heart and renal disease with (congestive) heart failure',
                    'Hypertensive heart disease with (congestive) heart failure', 
                    'Hypertensive heart disease without (congestive) heart failure', 'Insulin-dependent diabetes mellitus',
                    'Iron deficiency anaemia, unspecified', 'Malnutrition-related diabetes mellitus',
                    'Non-insulin-dependent diabetes mellitus', 'Other chronic obstructive pulmonary disease',
                    'Other iron deficiency anaemias', 'Other specified diabetes mellitus', 'Pulmonary oedema', 'Sideropenic dysphagia',
                    'Simple and mucopurulent chronic bronchitis', 'Status asthmaticus', 'Status epilepticus', 
                    'Unspecified chronic bronchitis', 'Unspecified dementia', 'Unspecified diabetes mellitus','Vascular dementia', 
                    'Vitamin B12 deficiency anaemia']

def categorise_level_description(value):
    if value in national_labels:
        return 'National'
    elif value in sex_labels:
        return 'Gender'
    elif value in age_groups:
        return 'Age'
    elif value in deprivation_labels:
        return 'Deprivation'
    elif value in known_regions:
        return 'Region'
    elif value in known_conditions:
        return 'Condition'
    else:
        return 'Umknown'

df['Category'] = df['Level description'].apply(categorise_level_description)


In [16]:
df['Percent unclassified'].replace(df['Percent unclassified'].median(), inplace=True)

/var/folders/c2/lbl69d113l3cxk_srs9x7_cc0000gn/T/ipykernel_48732/4102690978.py:1: FutureWarning: Series.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  df['Percent unclassified'].replace(df['Percent unclassified'].median(), inplace=True)
/var/folders/c2/lbl69d113l3cxk_srs9x7_cc0000gn/T/ipykernel_48732/4102690978.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Percent unclassified'].replace(df['Percent unclassified

In [17]:
df.isnull().sum()

Year                              0
Quarter                           0
Period of coverage                0
Breakdown                         0
Level                             0
Level description                 0
Indicator value                   0
Lower CI                          0
Upper CI                          0
Standardised ratio                0
Standardised ratio lower CI       0
Standardised ratio upper CI       0
Observed                          0
Population                        0
Expected                          0
Percent unclassified           8838
Category                          0
dtype: int64

In [18]:
(df == '*').sum()

Year                             0
Quarter                          0
Period of coverage               0
Breakdown                        0
Level                            0
Level description                0
Indicator value                362
Lower CI                       362
Upper CI                       362
Standardised ratio             362
Standardised ratio lower CI    362
Standardised ratio upper CI    362
Observed                       360
Population                       0
Expected                       349
Percent unclassified             0
Category                         0
dtype: int64

In [19]:
star_col = (df == '*').sum()
affected_cols = star_col[star_col > 0].index.tolist()
print('Columns with * values:, ', affected_cols)

Columns with * values:,  ['Indicator value', 'Lower CI', 'Upper CI', 'Standardised ratio', 'Standardised ratio lower CI', 'Standardised ratio upper CI', 'Observed', 'Expected']


In [20]:
for col in affected_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [21]:
for col in affected_cols:
    df[col].fillna(df[col].median(), inplace=True)

/var/folders/c2/lbl69d113l3cxk_srs9x7_cc0000gn/T/ipykernel_48732/2950632415.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
/var/folders/c2/lbl69d113l3cxk_srs9x7_cc0000gn/T/ipykernel_48732/2950632415.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always

In [22]:
print((df == '*').sum())
print(df.dtypes)

Year                           0
Quarter                        0
Period of coverage             0
Breakdown                      0
Level                          0
Level description              0
Indicator value                0
Lower CI                       0
Upper CI                       0
Standardised ratio             0
Standardised ratio lower CI    0
Standardised ratio upper CI    0
Observed                       0
Population                     0
Expected                       0
Percent unclassified           0
Category                       0
dtype: int64
Year                            object
Quarter                         object
Period of coverage              object
Breakdown                       object
Level                           object
Level description               object
Indicator value                float64
Lower CI                       float64
Upper CI                       float64
Standardised ratio             float64
Standardised ratio lower CI    float

In [23]:
# Saved the cleaned version

df.to_csv('../data/processed/emergency_readmission_cleaned.csv', index=False)

#### Inserting into PostgreSQL

In [24]:
insert_dataframe_to_db(df, 'readmission_cleaned')

Inserted data into table: readmission_cleaned
